### 경기도 내 공원 단계구분도

In [3]:
import numpy as np
import pandas as pd
import folium
import json

- 시군구별 공원면적 단계 구분도

In [4]:
gpark = pd.read_csv('data/경기도도시공원정보표준데이터.csv', encoding='euc-kr')
gpark.head(1)


,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명
0,11170-00048,녹색소공원,소공원,NaN,서울특별시 용산구 효창동 287-6,37.539217,126.962694,1569.9,NaN,NaN,NaN,NaN,NaN,2012-10-19,서울특별시 용산구청,02-2199-7605,2022-12-20,3020000,서울특별시 용산구


In [5]:
gpark = gpark[['소재지지번주소', '공원면적']]

gpark = gpark[gpark.소재지지번주소.str.find('경기도')>=0]   # 경기도만 추출

gpark.columns = ['주소', '공원면적']    # 컬럼명 변경
gpark.head()

,주소,공원면적
212,경기도 이천시 중리동 15,2526.0
213,경기도 이천시 중리동 210,3830.0
214,경기도 이천시 창전동 408,1757.0
215,경기도 이천시 창전동 414-13,1650.0
216,경기도 이천시 창전동 423,1650.0


In [6]:
gpark.공원면적 = gpark.공원면적.astype(int)   # 면적 타입을 정수로
gpark.head()

,주소,공원면적
212,경기도 이천시 중리동 15,2526
213,경기도 이천시 중리동 210,3830
214,경기도 이천시 창전동 408,1757
215,경기도 이천시 창전동 414-13,1650
216,경기도 이천시 창전동 423,1650


In [7]:
gpark.isna().sum()

주소      0
공원면적    0
dtype: int64

In [8]:
# 주소에서 시군구 이름 추출
g_list = []
for addr in gpark.주소:
    if addr.find('구 ') >=0:
        g_list.append(''.join(addr.split()[1:3]))
    else:
        g_list.append(addr.split()[1])

gpark['시군구'] = g_list
gpark.head()


,주소,공원면적,시군구
212,경기도 이천시 중리동 15,2526,이천시
213,경기도 이천시 중리동 210,3830,이천시
214,경기도 이천시 창전동 408,1757,이천시
215,경기도 이천시 창전동 414-13,1650,이천시
216,경기도 이천시 창전동 423,1650,이천시


In [9]:
# 시군구별 공원면적 합 구하기
gpark = gpark.pivot_table('공원면적', '시군구', aggfunc='sum')
gpark.head()

,공원면적
시군구,
가평군,511673
고양시덕양구,3287283
고양시일산동구,2722980
고양시일산서구,922486
과천시,315666


In [10]:
fname = 'data/경기도_geo_simple.json'   
geo_data = json.load(open(fname, encoding='utf-8'))

In [11]:
# 각 시군구별 평균 위치 구하기
def get_text_location(geo_str):
    gu_dict = {}
    for gu in geo_str['features']:
        for coord in gu['geometry']['coordinates']:
            geo = np.array(coord)
            gu_dict[gu['properties']['name']] = [np.mean(geo[:,1]), np.mean(geo[:,0])]
    return gu_dict

gu_dict = get_text_location(geo_data)

In [12]:
# 지도에 시군구 이름 표시
def show_name(m_pop, m_dict, m_map):
    for name in m_pop.index:
        folium.map.Marker(
            location=m_dict[name],
            icon=folium.DivIcon(icon_size=(80,20), icon_anchor=(20,0),
                                html=f'<div style="font-size: 10pt">{name}</div>')
        ).add_to(m_map)        

In [13]:
gmap = folium.Map([37.4, 127.1], zoom_start=9, tiles='Stamen Toner')

folium.Choropleth(
    geo_data=geo_data,
    data=gpark.공원면적,
    columns=[gpark.index, gpark.공원면적],
    fill_color='YlOrRd',
    key_on='properties.name'
).add_to(gmap)

show_name(gpark, gu_dict, gmap)

title_html = '<h3 align="center" style="font-size: 20px;">경기도 시군구별 공원면적</h3>'  
gmap.get_root().html.add_child(folium.Element(title_html))
gmap

- 시군구별 인당 공원면적 단계 구분도

In [14]:
# 경기도 시군구별 인구수 구하기
except_city = ['경기도 수원시', '경기도 고양시', '경기도 용인시', '경기도 성남시', '경기도 안산시', '경기도 안양시']


pop = pd.read_csv('../02.Pandas/data/주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명', '행정구역명', '총 인구수']]
pop = pop[((pop.행정구역구분명 == '시군') | (pop.행정구역구분명 == '구')) & ~(pop['행정구역명'].str.strip().isin(except_city))]
pop = pop[['행정구역명', '총 인구수']]
pop.head()


,행정구역명,총 인구수
1,경기도 가평군,62197
9,경기도 고양시 덕양구,487874
31,경기도 고양시 일산동구,296590
44,경기도 고양시 일산서구,290738
56,경기도 과천시,77775


In [15]:
pop['시군구'] = pop.행정구역명.apply(lambda x: ''.join(x.split()[1:]))
pop.head()

,행정구역명,총 인구수,시군구
1,경기도 가평군,62197,가평군
9,경기도 고양시 덕양구,487874,고양시덕양구
31,경기도 고양시 일산동구,296590,고양시일산동구
44,경기도 고양시 일산서구,290738,고양시일산서구
56,경기도 과천시,77775,과천시


In [16]:
pop = pop[['시군구', '총 인구수']]
pop.set_index('시군구', inplace=True)
pop.columns = ['인구']
pop.head()

,인구
시군구,
가평군,62197
고양시덕양구,487874
고양시일산동구,296590
고양시일산서구,290738
과천시,77775


In [17]:
ggpark = pd.merge(gpark, pop, left_index=True, right_index=True)
ggpark.head()

,공원면적,인구
시군구,,
가평군,511673,62197
고양시덕양구,3287283,487874
고양시일산동구,2722980,296590
고양시일산서구,922486,290738
과천시,315666,77775


In [18]:
ggpark['인당공원면적'] = ggpark.인구/ggpark.공원면적 * 100

In [19]:
ggpark.head()

,공원면적,인구,인당공원면적
시군구,,,
가평군,511673,62197,12.155615
고양시덕양구,3287283,487874,14.841253
고양시일산동구,2722980,296590,10.892111
고양시일산서구,922486,290738,31.516793
과천시,315666,77775,24.638384


In [20]:
gmap = folium.Map([37.4, 127.1], zoom_start=9, tiles='Stamen Toner')

folium.Choropleth(
    geo_data=geo_data,
    data=ggpark.인당공원면적,
    columns=[ggpark.index, ggpark.인당공원면적],
    fill_color='YlOrRd',
    key_on='properties.name'
).add_to(gmap)

show_name(ggpark, gu_dict, gmap)

title_html = '<h3 align="center" style="font-size: 20px;">경기도 시군구별 인당 공원면적</h3>'  
gmap.get_root().html.add_child(folium.Element(title_html))
gmap